In [1]:
import pandas as pd

output_correct = pd.DataFrame()
output_error = pd.DataFrame()

In [2]:
input_ = pd.read_excel('./1.manufacturer.xlsx',
                       header=0,
                       dtype=str).fillna('')

print('总数量：' + str(len(input_)))
input_['Manufacturer'].tolist()

总数量：553


['3D',
 '3M',
 'AAE',
 'ACC',
 'ACCEL',
 'ACCUVISION',
 'ACDELCO',
 'ACI',
 'ACME AUTO',
 'ADVAN-TECH',
 'ADVICS',
 'AEM INDUCTION',
 'AGILITY',
 'AGS',
 'AIMCO',
 'AIR PRO',
 'AIRAID',
 'AIRQUALITEE',
 'AIRTEX',
 'AISIN',
 'AJUSA',
 'AKEBONO',
 "AL'S LINER",
 'AL-KO',
 'ALBANY',
 'ALLPARTS',
 'ALO',
 'ALTROM',
 'AMERICAN COMPONENTS',
 'AMERICAN REMANUFACTURERS INC.',
 'AMS AUTOMOTIVE',
 'ANCHOR',
 'ANCO',
 'ANSA',
 'AP / EASTERN',
 'APEX',
 'API',
 'APWI',
 'ARIES',
 'ARMORMARK',
 'ARNOTT',
 'ARP',
 'ARROW AUTOMOTIVE',
 'ATE',
 'ATP',
 'ATSCO',
 'AUS INJECTION INC.',
 'AUTO 7',
 'AUTO EXTRA',
 'AUTO EXTRA/CHASSIS RITE',
 'AUTO PRIDE',
 'AUTO-TUNE',
 'AUTOBEST',
 'AUTODRIVE',
 'AUTOLINE',
 'AUTOLITE',
 'AUTOMETER',
 'AUTOMOTIVE UNDERCAR PRODUCTS',
 'AUTOPOWER',
 'AUTOSPECIALTY',
 'AUTOSPECIALTY/KELSEY-HAYES',
 'AUTOTECH ENGINEERING',
 'AUTOTECNICA',
 'AUTOTEMP',
 'AUVECO',
 'AVS',
 'B&B',
 "B'LASTER",
 'BALKAMP',
 'BANDO',
 'BB BRAKES',
 'BBB INDUSTRIES',
 'BCA',
 'BECK/ARNLEY',
 'BELD

In [3]:
crawler_status = 'error'

crawler_status

'error'

In [4]:
import requests

import sys
sys.path.append('../../00.Tools')
import UA
import Proxy

In [5]:
a = 0

data = {'func': 'partsearchgetpartgroupsdd',
        'mfr': input_.loc[a, 'Manufacturer'],
        'api_json_request': '1'}

resp = requests.post('https://www.rockauto.com/catalog/catalogapi.php',
                     data=data,
                     headers=UA.get_User_Agent_Requests(),
                     proxies=Proxy.get_Proxy_Requests()).text

resp

'{"html_replace_sections":{"ddrepl[partgroup_][container]":"<span id=\\"ddrepl[partgroup_][container]\\" class=\\"ra-vertical-dropdown-replacement-c\\"><select id=\\"partgroup_\\" name=\\"partsearch[partgroup][]\\" onchange = \\"clb.ChangePartGroupOption(this, null);;\\" tabindex=\\"0\\" autocomplete = \\"none\\"><option value=\\"\\"  selected >All<\\/option><option value=\\"Body &amp; Lamp Assembly\\"  >Body &amp; Lamp Assembly<\\/option><option value=\\"Interior\\"  >Interior<\\/option><\\/select><\\/span>"},"collected_javascript":"\\/**UQ:22F915B3**\\/\\nwindow.top.parent.window[\\"_nck\\"] = \\"8LVLUxTaILBT+tLxkg8+3Ag7\\\\\\/0bbOhvn7P3rkFbL80\\\\\\/oQB6RvzNfa5dcHvATqXAMxw5whzaDEdxk+27zeij67BjKru6WqTdNqG6tiEsD1EZhbTv0XForkbYp+nLaIKxDXfb4DjpqS2ZFbzKKm02sKiciIizHymck21OTzzFipS8=\\";\\n"}'

In [6]:
import json

In [7]:
json_data = json.loads(resp)

json_data

{'html_replace_sections': {'ddrepl[partgroup_][container]': '<span id="ddrepl[partgroup_][container]" class="ra-vertical-dropdown-replacement-c"><select id="partgroup_" name="partsearch[partgroup][]" onchange = "clb.ChangePartGroupOption(this, null);;" tabindex="0" autocomplete = "none"><option value=""  selected >All</option><option value="Body &amp; Lamp Assembly"  >Body &amp; Lamp Assembly</option><option value="Interior"  >Interior</option></select></span>'},
 'collected_javascript': '/**UQ:22F915B3**/\nwindow.top.parent.window["_nck"] = "8LVLUxTaILBT+tLxkg8+3Ag7\\/0bbOhvn7P3rkFbL80\\/oQB6RvzNfa5dcHvATqXAMxw5whzaDEdxk+27zeij67BjKru6WqTdNqG6tiEsD1EZhbTv0XForkbYp+nLaIKxDXfb4DjpqS2ZFbzKKm02sKiciIizHymck21OTzzFipS8=";\n'}

In [8]:
from bs4 import BeautifulSoup
from lxml import etree

In [9]:
soup = BeautifulSoup(json_data['html_replace_sections']['ddrepl[partgroup_][container]'], 'lxml')
html = etree.HTML(str(soup))

with open('./category.html', 'w', encoding='UTF-8') as file:
    file.write(str(soup))

print(soup.prettify())

<html>
 <body>
  <span class="ra-vertical-dropdown-replacement-c" id="ddrepl[partgroup_][container]">
   <select autocomplete="none" id="partgroup_" name="partsearch[partgroup][]" onchange="clb.ChangePartGroupOption(this, null);;" tabindex="0">
    <option selected="" value="">
     All
    </option>
    <option value="Body &amp; Lamp Assembly">
     Body &amp; Lamp Assembly
    </option>
    <option value="Interior">
     Interior
    </option>
   </select>
  </span>
 </body>
</html>



In [10]:
list_category = [category.strip() for category in html.xpath('//select[@id="partgroup_"]/option/text()')[1:]]

list_category

['Body & Lamp Assembly', 'Interior']

In [11]:
df_temp = pd.DataFrame({'Manufacturer_No.': int(input_.loc[a, 'Manufacturer_No.']),
                        'Manufacturer': input_.loc[a, 'Manufacturer'],
                        'Category_No.': [i+1 for i in range(len(list_category))],
                        'Category': list_category})

output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

len(output_correct)

2

In [12]:
crawler_status = 'ok'

crawler_status

'ok'

In [13]:
if crawler_status == 'error':
    df_temp = pd.DataFrame([{'Manufacturer_No.': int(input_.loc[a, 'Manufacturer_No.']),
                             'Manufacturer': input_.loc[a, 'Manufacturer']}])
    
    output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

len(output_error)

0

In [14]:
output_correct = output_correct.drop_duplicates(ignore_index=True)
output_correct = output_correct.sort_values(by=['Manufacturer_No.', 'Category_No.'], ascending=[True, True]).reset_index(drop=True)
output_correct.to_excel('./test_category.xlsx', index=False)

output_correct

,Manufacturer_No.,Manufacturer,Category_No.,Category
0,1,3D,1,Body & Lamp Assembly
1,1,3D,2,Interior


In [15]:
if len(output_error) != 0:
    output_error = output_error.drop_duplicates(ignore_index=True)
    output_error = output_error.sort_values(by=['Manufacturer_No.'], ascending=[True]).reset_index(drop=True)
    output_error.to_excel('./test_category_error.xlsx', index=False)

output_error

""
